In [46]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
#set paths
mfcc_path = 'mfccs_quantized'
train_path = os.path.join(mfcc_path, 'training')
val_path = os.path.join(mfcc_path, 'validation')
test_path = os.path.join(mfcc_path, 'test')

#set constants
n_mfcc = 20
n_dim = 126
reject_label = 'UNKNOWN'

In [3]:
all_words = list(os.listdir(train_path))

In [4]:
#20 keywords to recognize
keywords = ['up','down','left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 
             'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']

#all other words will be rejection class
rejection_words = [i for i in all_words if i not in keywords]

In [5]:
#check how many MFCCs don't have shape (20, 126)
count = 0
for i in [test_path, val_path, train_path]:
    for j in os.listdir(train_path):
        print(j)
        data_path = os.path.join(i, j)
        for k in os.listdir(data_path):
            file = os.path.join(data_path, k)
            mfcc = np.load(file)
            if(mfcc.shape != (n_mfcc, n_dim)):
                count += 1
count

backward
bed
bird
cat
dog
down
eight
five
follow
forward
four
go
happy
house
learn
left
marvin
nine
no
off
on
one
right
seven
sheila
six
stop
three
tree
two
up
visual
wow
yes
zero
backward
bed
bird
cat
dog
down
eight
five
follow
forward
four
go
happy
house
learn
left
marvin
nine
no
off
on
one
right
seven
sheila
six
stop
three
tree
two
up
visual
wow
yes
zero
backward
bed
bird
cat
dog
down
eight
five
follow
forward
four
go
happy
house
learn
left
marvin
nine
no
off
on
one
right
seven
sheila
six
stop
three
tree
two
up
visual
wow
yes
zero


0

In [43]:
train_len = 0
val_len = 0
test_len = 0

for i in [train_path, val_path, test_path]:
    for j in keywords:
        data_path = os.path.join(i, j)
        count += len(os.listdir(data_path))
        
    if (i == train_path):
        train_len = count
    elif (i == val_path):
        val_len = count
    else:
        test_len = count
    count = 0
print('Train length', train_len)
print('Validation length', val_len)
print('Test length', test_len)

Train length 61927
Validation length 7346
Test length 8181


In [68]:
#initialize arrays for the features, we will have to drop the first entries later
X_train = np.zeros(shape = (train_len, n_mfcc, n_dim))
X_val = np.zeros(shape = (val_len, n_mfcc, n_dim))
X_test = np.zeros(shape = (test_len, n_mfcc, n_dim))

# y_train = np.zeros(shape = (train_len))
# y_val = np.zeros(shape = (val_len))
# y_test = np.zeros(shape = (test_len))
y_train = np.empty(shape=(train_len), dtype='<U10')
y_val = np.empty(shape=(val_len), dtype='<U10')
y_test = np.empty(shape=(test_len), dtype='<U10')

In [69]:
start = time.time()

#loop through train, validation, and test data
for i in [train_path, val_path, test_path]:
    idx = 0
    
    #loop through all the word directories
    for j in keywords:
        data_path = os.path.join(i, j)
        print(j)
        #load mfcc for each file in directory
        for k in os.listdir(data_path):
            file = os.path.join(data_path, k)
            mfcc = np.load(file)
            
            #append mfcc to train, validation, or test
            if (i == train_path):
#                 X_train = np.append(X_train, mfcc, axis = 0)
#                 y_train = np.append(y_train, j)
                X_train[idx] = mfcc
                y_train[idx] = j
                    
            elif (i == test_path):
#                 X_test = np.append(X_test, mfcc, axis = 0)
#                 y_test = np.append(y_test, j)
                X_test[idx] = mfcc
                y_test[idx] = j
            elif (i == val_path):
#                 X_val = np.append(X_val, mfcc, axis = 0)
#                 y_val = np.append(y_val, j)
                X_val[idx] = mfcc
                y_val[idx] = j
            idx += 1
    
end = time.time()
print(end - start, 'seconds')
#drop the first n_mfcc entries because they're zeros from initialization
# X_train = X_train[n_mfcc:]                
# X_val = X_val[n_mfcc:]
# X_test = X_test[n_mfcc:]

#drop the first entry because they're also zero from initialization
# y_train = y_train[1:]
# y_val = y_val[1:]
# y_test = y_test[1:]

#get length of arrays in terms of mfcc size
train_length = int(len(X_train) / n_mfcc)
val_length = int(len(X_val) / n_mfcc)
test_length = int(len(X_test) / n_mfcc)


#reshape arrays to be (n, 20, 32) where n is number of samples in train, validation, or test
# X_train = np.reshape(X_train, newshape = (train_length, n_mfcc, n_dim))
# X_val = np.reshape(X_val, newshape = (val_length, n_mfcc, n_dim))
# X_test = np.reshape(X_test, newshape = (test_length, n_mfcc, n_dim))

up
down
left
right
stop
go
yes
no
on
off
one
two
three
four
five
six
seven
eight
nine
zero
up
down
left
right
stop
go
yes
no
on
off
one
two
three
four
five
six
seven
eight
nine
zero
up
down
left
right
stop
go
yes
no
on
off
one
two
three
four
five
six
seven
eight
nine
zero
184.2460663318634 seconds


In [70]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)

(61927, 20, 126)
(61927,)
(7346, 20, 126)
(7346,)
(8181, 20, 126)
(8181,)


These shapes correspond to the number of MFCCs generated previously and are ready to be put into a neural network.

In [74]:
#save files
cwd = os.getcwd()
np.save(os.path.join(cwd, 'X_train_quant'), X_train)
np.save(os.path.join(cwd, 'y_train_quant'), y_train)
np.save(os.path.join(cwd, 'X_val_quant'), X_val)
np.save(os.path.join(cwd, 'y_val_quant'), y_val)
np.save(os.path.join(cwd, 'X_test_quant'), X_test)
np.save(os.path.join(cwd, 'y_test_quant'), y_test)